# 市中残存額分析

銘柄ごとの市中残存額（累積発行額 - 日銀保有額）を可視化します。

## 市中残存額の定義
- **累積発行額**: その日付までの発行入札で割り当てられた総額
- **日銀保有額**: その日に一番近い日に発表された日銀保有量
- **市中残存額** = 累積発行額 - 日銀保有額

## 注意事項
- 日銀は発行入札だけでなく、**市場からの買入オペ**でも国債を購入します
- そのため、日銀保有額が累積発行額を上回る（市中残存額がマイナスになる）ことがあります
- これは「日銀が発行額以上に保有している」状態を示しています

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from dotenv import load_dotenv
from supabase import create_client

# 日本語フォント設定
plt.rcParams['font.family'] = 'Hiragino Sans'
plt.rcParams['axes.unicode_minus'] = False

# Supabase接続
load_dotenv()
url = os.getenv('SUPABASE_URL')
key = os.getenv('SUPABASE_KEY')
supabase = create_client(url, key)

print('Supabase接続完了')

## データ取得関数

In [ ]:
def get_bond_info(bond_code: str) -> dict:
    """銘柄の基本情報を取得"""
    result = supabase.table('bond_data') \
        .select('bond_code, bond_name, coupon_rate, due_date') \
        .eq('bond_code', bond_code) \
        .limit(1) \
        .execute()
    
    if result.data:
        return result.data[0]
    return None


def get_auction_data(bond_code: str) -> pd.DataFrame:
    """発行入札データを取得（累積発行額計算用）"""
    result = supabase.table('bond_auction') \
        .select('auction_date, allocated_amount') \
        .eq('bond_code', bond_code) \
        .order('auction_date') \
        .execute()
    
    if not result.data:
        return pd.DataFrame()
    
    df = pd.DataFrame(result.data)
    df['auction_date'] = pd.to_datetime(df['auction_date'])
    df['allocated_amount'] = pd.to_numeric(df['allocated_amount'], errors='coerce')
    return df


def get_boj_holdings(bond_code: str) -> pd.DataFrame:
    """日銀保有データを取得"""
    result = supabase.table('boj_holdings') \
        .select('data_date, face_value') \
        .eq('bond_code', bond_code) \
        .order('data_date') \
        .execute()
    
    if not result.data:
        return pd.DataFrame()
    
    df = pd.DataFrame(result.data)
    df['data_date'] = pd.to_datetime(df['data_date'])
    df['face_value'] = pd.to_numeric(df['face_value'], errors='coerce')
    return df


print('データ取得関数を定義しました')

## 市中残存額計算関数

In [ ]:
def calculate_market_outstanding(bond_code: str) -> pd.DataFrame:
    """
    市中残存額を計算
    
    Returns:
        DataFrame with columns: date, cumulative_issuance, boj_holding, market_outstanding
    """
    # データ取得
    auction_df = get_auction_data(bond_code)
    boj_df = get_boj_holdings(bond_code)
    
    if auction_df.empty:
        print(f'警告: {bond_code} の発行データがありません')
        return pd.DataFrame()
    
    # 累積発行額を計算
    auction_df = auction_df.sort_values('auction_date')
    auction_df['cumulative_issuance'] = auction_df['allocated_amount'].cumsum()
    
    # 日付範囲を決定（発行日から最新の日銀データ日まで）
    start_date = auction_df['auction_date'].min()
    if not boj_df.empty:
        end_date = max(auction_df['auction_date'].max(), boj_df['data_date'].max())
    else:
        end_date = auction_df['auction_date'].max()
    
    # 日次の日付範囲を作成
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    result_df = pd.DataFrame({'date': date_range})
    
    # 累積発行額をマージ（前方補完）
    auction_for_merge = auction_df[['auction_date', 'cumulative_issuance']].copy()
    auction_for_merge = auction_for_merge.rename(columns={'auction_date': 'date'})
    result_df = result_df.merge(auction_for_merge, on='date', how='left')
    result_df['cumulative_issuance'] = result_df['cumulative_issuance'].ffill()
    result_df['cumulative_issuance'] = result_df['cumulative_issuance'].fillna(0)
    
    # 日銀保有額をマージ（前方補完 - 一番近い過去の日付のデータを使用）
    if not boj_df.empty:
        boj_for_merge = boj_df[['data_date', 'face_value']].copy()
        boj_for_merge = boj_for_merge.rename(columns={'data_date': 'date', 'face_value': 'boj_holding'})
        result_df = result_df.merge(boj_for_merge, on='date', how='left')
        result_df['boj_holding'] = result_df['boj_holding'].ffill()
        result_df['boj_holding'] = result_df['boj_holding'].fillna(0)
    else:
        result_df['boj_holding'] = 0
    
    # 市中残存額を計算
    result_df['market_outstanding'] = result_df['cumulative_issuance'] - result_df['boj_holding']
    
    # 発行前の期間を除外
    result_df = result_df[result_df['cumulative_issuance'] > 0]
    
    return result_df


print('市中残存額計算関数を定義しました')

## 可視化関数

In [ ]:
def plot_market_outstanding(bond_code: str, figsize=(14, 8)):
    """
    市中残存額をグラフで表示
    """
    # 銘柄情報取得
    bond_info = get_bond_info(bond_code)
    if bond_info:
        title = f"{bond_info['bond_name']} ({bond_code})\nクーポン: {bond_info['coupon_rate']}% / 償還日: {bond_info['due_date']}"
    else:
        title = f"銘柄コード: {bond_code}"
    
    # データ計算
    df = calculate_market_outstanding(bond_code)
    
    if df.empty:
        print(f'データがありません: {bond_code}')
        return
    
    # グラフ作成
    fig, ax = plt.subplots(figsize=figsize)
    
    # 線グラフで表示（マイナス値も対応）
    ax.plot(df['date'], df['cumulative_issuance'], 'g-', linewidth=2, label='累積発行額')
    ax.plot(df['date'], df['boj_holding'], 'r-', linewidth=2, label='日銀保有額')
    ax.plot(df['date'], df['market_outstanding'], 'b-', linewidth=2, label='市中残存額')
    
    # ゼロラインを追加
    ax.axhline(y=0, color='gray', linestyle='--', linewidth=0.8, alpha=0.5)
    
    # 面グラフ（市中残存額がプラスの部分のみ塗りつぶし）
    ax.fill_between(df['date'], 0, df['market_outstanding'], 
                    where=(df['market_outstanding'] >= 0),
                    alpha=0.2, color='blue', label='_nolegend_')
    ax.fill_between(df['date'], 0, df['market_outstanding'], 
                    where=(df['market_outstanding'] < 0),
                    alpha=0.2, color='red', label='_nolegend_')
    
    # 装飾
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel('日付', fontsize=12)
    ax.set_ylabel('金額（億円）', fontsize=12)
    ax.legend(loc='upper left', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # X軸の日付フォーマット
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
    plt.xticks(rotation=45)
    
    # 最新値を表示
    latest = df.iloc[-1]
    stats_text = f"最新日: {latest['date'].strftime('%Y-%m-%d')}\n"
    stats_text += f"累積発行額: {latest['cumulative_issuance']:,.0f}億円\n"
    stats_text += f"日銀保有額: {latest['boj_holding']:,.0f}億円\n"
    stats_text += f"市中残存額: {latest['market_outstanding']:,.0f}億円\n"
    if latest['cumulative_issuance'] > 0:
        boj_ratio = latest['boj_holding'] / latest['cumulative_issuance'] * 100
        stats_text += f"日銀保有比率: {boj_ratio:.1f}%"
    
    ax.text(0.98, 0.02, stats_text, transform=ax.transAxes, fontsize=10,
            verticalalignment='bottom', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()
    
    return df


print('可視化関数を定義しました')

## 銘柄検索ヘルパー

In [ ]:
def search_bonds(keyword: str = None, bond_type: str = None, limit: int = 20):
    """
    銘柄を検索
    
    Args:
        keyword: 銘柄名に含まれるキーワード
        bond_type: 国債種別（'2年債', '5年債', '10年債', '20年債', '30年債', '40年債'）
        limit: 取得件数
    """
    # bond_auctionから銘柄一覧を取得
    result = supabase.table('bond_auction') \
        .select('bond_code, issue_number, issue_date, maturity_date, coupon_rate, allocated_amount') \
        .order('issue_date', desc=True) \
        .limit(500) \
        .execute()
    
    if not result.data:
        print('データがありません')
        return pd.DataFrame()
    
    df = pd.DataFrame(result.data)
    
    # 種別コードから国債種別を判定
    type_mapping = {
        '042': '2年債',
        '045': '5年債',
        '067': '10年債',
        '069': '20年債',
        '068': '30年債',
        '054': '40年債',
        '046': '物価連動債',
    }
    df['bond_type'] = df['bond_code'].str[-3:].map(type_mapping).fillna('その他')
    
    # フィルタ
    if bond_type:
        df = df[df['bond_type'] == bond_type]
    
    # 重複除去（銘柄コードでユニーク）
    df = df.drop_duplicates(subset='bond_code')
    
    return df.head(limit)


# 使用例を表示
print('銘柄検索例:')
print('  search_bonds(bond_type="10年債")  # 10年債を検索')
print('  search_bonds(bond_type="2年債", limit=10)  # 2年債を10件検索')

---
## 使用例

In [ ]:
# 10年債の銘柄一覧を確認
bonds_10y = search_bonds(bond_type='10年債', limit=10)
display(bonds_10y)

In [ ]:
# 特定の銘柄の市中残存額をグラフ表示
# 例: 10年債の最新銘柄
if not bonds_10y.empty:
    sample_bond_code = bonds_10y.iloc[0]['bond_code']
    print(f'分析対象: {sample_bond_code}')
    df = plot_market_outstanding(sample_bond_code)

In [ ]:
# 任意の銘柄コードを指定して分析
# bond_code = '003720067'  # 例: 10年372回
# df = plot_market_outstanding(bond_code)

## 複数銘柄の比較

In [ ]:
def compare_market_outstanding(bond_codes: list, figsize=(14, 8)):
    """
    複数銘柄の市中残存額を比較
    """
    fig, ax = plt.subplots(figsize=figsize)
    
    for bond_code in bond_codes:
        df = calculate_market_outstanding(bond_code)
        if df.empty:
            continue
        
        bond_info = get_bond_info(bond_code)
        label = bond_info['bond_name'] if bond_info else bond_code
        
        ax.plot(df['date'], df['market_outstanding'], linewidth=2, label=label)
    
    ax.set_title('市中残存額の比較', fontsize=14, fontweight='bold')
    ax.set_xlabel('日付', fontsize=12)
    ax.set_ylabel('市中残存額（億円）', fontsize=12)
    ax.legend(loc='best', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()


# 使用例（コメントを外して実行）
# compare_market_outstanding(['003720067', '003710067', '003700067'])